# 1. 导入相关工具包

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# 2. 数据读取及预处理

In [2]:
df = pd.read_csv('/home/mw/input/credit_card8849/card_transdata.csv')
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


distance_from_home	 ：离发生交易的位置的距离  
distance_from_last_transaction	 ：距离上一次交易发生的距离  
ratio_to_median_purchase_price	 ：购买价格与中位购买价格的比值  
repeat_retailer	 ：交易是否来自同一零售商  
used_chip	 ：是否通过芯片（信用卡）进行的交易  
used_pin_number	 ：交易是否用密码进行  
online_order	 交易是否为在线订单  
fraud：交易是否具有欺诈性

In [3]:
print("数据维度：", df.shape)

数据维度： (1000000, 8)


In [4]:
print("缺失值：", df.isnull().sum())

缺失值： distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64


In [5]:
print("重复值：", df.duplicated().sum())

重复值： 0


In [6]:
df.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,26.628792,5.036519,1.824182,0.881536,0.350399,0.100608,0.650552,0.087403
std,65.390784,25.843093,2.799589,0.323157,0.477095,0.300809,0.476796,0.282425
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.878008,0.296671,0.475673,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.967760,0.998650,0.997717,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.743985,3.355748,2.096370,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


# 3. 探索性分析

## 3.1 数据分布详情

In [7]:
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

In [8]:
n_sample = df.shape[0]
n_0_sample = df.fraud.value_counts()[0]
n_1_sample = df.fraud.value_counts()[1]
print("样本个数：{}；0占{}；1占{}".format(n_sample, n_0_sample/n_sample, n_1_sample/n_sample))

样本个数：1000000；0占0.912597；1占0.087403


In [9]:
df_t = df.fraud.value_counts().reset_index()
df_t.replace({0:'非欺诈', 1:'欺诈'}, inplace=True)

fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
sns.barplot(x = "index", y="fraud", data = df_t)
plt.xlabel("")
plt.ylabel('count', fontdict={'fontsize':12})

ax2 = fig.add_subplot(1, 2, 2)
plt.pie(x = df_t['fraud'], labels=df_t['index'], autopct='%1.1f%%', explode=[0.1, 0],
    startangle=90, counterclock=False, wedgeprops={'linewidth':1, 'edgecolor':'black'})
plt.axis('square')
plt.show()

<Figure size 720x288 with 2 Axes>

数据集中存在“欺诈”的交易占比较少，数据分布极不平均，为保证后续构建的模型的稳定性，故在之后进行“过采样”的处理

## 3.2 数据内部相关热力图

In [10]:
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap = 'coolwarm', linewidth=0.5)

<Figure size 432x288 with 2 Axes>

针对交易是否具有欺诈性，最重要的特征是“购买价格与中位购买价格的比率”（0.46），其次是“交易是否为在线订单”（0.19）与“离发生交易的位置的距离”（0.19）

## 3.3 对目标变量进行过采样

### 过采样与欠采样（数据重采样）：  
SMOTE过采样：对少数类别样本a，随机选择一个最邻近的样本b，然后从a与b的连线上随机选择一个点c作为新的少数类样本  
TomekLinks欠采样：通过删除多数类别中的一些样本进行欠采样的方法

### 这两个方法的优缺点：  
1. 过采样：  
	（1）优点：  
			改善分类性能：通过增加少数类样本的数量，使分类模型更好地学习少数类的特征，并提高对少数类样本的分类性能。  
			保留信息：过采样方法通过合成新的样本而不是仅仅复制已有样本，从而可以引入一定程度的新信息，丰富少数类样本的多样性。  
			不引入偏见：过采样在合成新的少数类样本时并不依赖于多数类样本，因此不会引入对多数类的任何偏见。  
	（2）缺点：  
			过拟合风险：由于增加了少数类样本的数量，可能导致模型对少数类样本过度拟合，从而降低其在未见样本上的泛化能力。  
			增加计算成本：过采样方法需要合成新的样本，这会增加数据集的规模，进而增加训练和预测的计算成本。  
			引入噪声：在合成新的少数类样本时，过采样方法可能引入一定程度的噪声，这可能对模型的性能产生不利影响。  
2. 欠采样：  
  （1）优点：  
			减少计算成本：通过删除大多数类的一部分样本来平衡类别分布，减少了训练样本的数量，从而可能减少模型训练时间。  
			避免引入额外噪声：只删除了原始数据，不会引入额外的噪声。  
	（2）缺点：  
			损失信息：可能会损失大多数类的关键信息，导致模型性能下降。  
			引入选择性偏差：由于删除了部分大多数类的样本，可能引入选择性偏差。

### 是否需要做重采样：  
1. 看类别的不平衡程度  
2. 看模型的性能效果  
3. 看数据集的大小

In [11]:
X = df.iloc[:,:-1]  # 特征列
y = df.iloc[:,-1]  # 目标列

from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)
# 使用SMOTE过采样时，X和y作为输入，SMOTE会根据少数类别的样本生成新的合成样本，以使两个类别样本数量更加平衡

In [12]:
n_sample_new = X_resampled.shape[0]
n_0_sample = y_resampled.value_counts()[0]
n_1_sample = y_resampled.value_counts()[1]
print("样本个数：{}；0占{}；1占{}".format(n_sample_new, n_0_sample/n_sample_new, n_1_sample/n_sample_new))

样本个数：1825194；0占0.5；1占0.5


# 4. 划分训练集和测试集

## 4.1 未进行过采样

In [13]:
# 未进行重采样
from sklearn.model_selection import train_test_split
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, test_size=0.2, random_state=2024)

## 4.2 已进行过采样

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=2024)

# 5. 模型训练

## 5.1 逻辑回归

### 5.1.1 未进行过采样

In [15]:
from sklearn.linear_model import LogisticRegression as LR
model_LR = LR(penalty = 'l2')
model_LR.fit(X_train0, y_train0)

LogisticRegression()

In [16]:
model_LR.score(X_test0, y_test0)

0.957555

### 5.1.2 已进行过采样

In [17]:
from sklearn.linear_model import LogisticRegression as LR
model_LR = LR(penalty = 'l2')
model_LR.fit(X_train, y_train)

LogisticRegression()

### 5.1.3 模型得分与ROC曲线

In [18]:
model_LR.score(X_test, y_test)

0.9429814348603848

做五折交叉验证：将数据划分成五个大小相同的份，循环5次，每次取4份做训练集，1份做测试集  
使用4份训练集对模型进行训练，并在验证集上进行评估，可以计算模型的性能指标  
完成五次循环后，可以将五次循环中得到的性能指标取平均值，作为模型性能的最终评估结果

In [19]:
from sklearn.model_selection import cross_val_score as CVS
CVS(model_LR, X_train, y_train, cv=5).mean()

0.9416931764093537

In [22]:
from sklearn import metrics 
import matplotlib.pyplot as plt

sklearn_predict = model_LR.predict(X_test)

# 绘制ROC曲线
fpr, tpr, _ = metrics.roc_curve(y_test, sklearn_predict)
plt.plot(fpr, tpr, color = 'red')
plt.stackplot(fpr, tpr, color = 'steelblue')
plt.plot([0,1],[0,1], linestyle = '--', color = 'black')
plt.text(0.6,0.4,'AUC=%.3f' % metrics.auc(fpr,tpr), fontdict = dict(size = 18))
plt.show()

<Figure size 432x288 with 1 Axes>

### 分析  
在逻辑回归模型中，未进行过采样的数据集在该模型上的得分为0.957555，进行过采样之后的数据集在该模型上的得分为0.9407624938705179，  
分数降低可能是因为原本数据集的数量有一百万条，其中类别较少的数据有87403条，有很多数据，所以在进行过采样之后，加入了新的合成样本，预测得分下降了  
但是分数下降不多，而且数据集分布极其不均衡，所以后面的两个模型还是选择进行过采样之后的数据进行训练。

## 5.2 XGBoost模型

In [23]:
from xgboost import XGBClassifier as XGBC
model_xgb = XGBC(n_estimators=100, max_depth=3)
model_xgb.fit(X_train, y_train)

XGBClassifier()

### 5.2.1 模型得分

In [24]:
model_xgb.score(X_test, y_test)

0.9998356340007506

In [25]:
CVS(model_xgb, X_train, y_train, cv=5).mean()

0.9997233170451082

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

y_test_pred = model_xgb.predict(X_test)
auc = accuracy_score(y_test, y_test_pred)
print("准确率为：{:.2f}%".format(auc))

recall = recall_score(y_test, y_test_pred)
print("召回率为：{:.2f}%".format(recall))

pre = precision_score(y_test, y_test_pred)
print("精确率为：{:.2f}%".format(pre))

f1 = f1_score(y_test, y_test_pred)
print("F1-score为{:.2f}%".format(f1))

准确率为：1.00%
召回率为：1.00%
精确率为：1.00%
F1-score为1.00%


### 5.2.2 模型的ROC曲线

In [27]:
# 计算绘图数据
y_pred = model_xgb.predict(X_test)
fpr,tpr,threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr,tpr)
# 绘图
plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.5, edgecolor = 'black')
plt.plot(fpr, tpr, color='black', lw = 1)
plt.plot([0,1],[0,1], color = 'red', linestyle = '--')
plt.text(0.5,0.3,'ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.show()

<Figure size 432x288 with 1 Axes>

### 5.2.3 变量的重要性程度可视化

In [28]:
model_xgb.feature_importances_
# 变量的重要性程度值
importance = model_xgb.feature_importances_
# 构建含序列用于绘图
Impt_Series = pd.Series(importance, index = X_train.columns)
# 对序列排序绘图
Impt_Series.sort_values(ascending = True).plot(kind='barh')
plt.show()

<Figure size 432x288 with 1 Axes>

## 5.3 朴素贝叶斯模型

In [29]:
from sklearn.naive_bayes import GaussianNB 
# 创建朴素贝叶斯分类器  
gnb = GaussianNB()  
# 使用训练数据拟合模型  
gnb.fit(X_train, y_train)  

GaussianNB()

In [30]:
  # 使用模型进行预测  
y_pred = gnb.predict(X_test) 

In [31]:
# result_df = X_test.copy()
# result_df['fraud'] = y_test
# result_df['fraud_pred'] = y_pred
# result_df.head()

### 5.3.1 模型分类报告和准确率

In [32]:
from sklearn import metrics 
# 打印分类报告和准确率  
print("Classification report for Naive Bayes classifier on dataset:\n%s\n"  
      % (metrics.classification_report(y_test, y_pred)))  
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification report for Naive Bayes classifier on dataset:
              precision    recall  f1-score   support

         0.0       0.95      0.61      0.74    182890
         1.0       0.71      0.97      0.82    182149

    accuracy                           0.79    365039
   macro avg       0.83      0.79      0.78    365039
weighted avg       0.83      0.79      0.78    365039


Accuracy: 0.7862529757094447


# 结论  
（经过五折交叉验证）LR模型得分：0.9409610623529693  
（经过五折交叉验证）XGBoost模型：0.9998061849598159  
朴素贝叶斯模型：0.7866145809077935